In [1]:
#export
"""Reliable websocket client and server handle functions"""
import k1lib, math, numpy as np, random, base64, json, time; import k1lib.cli as cli; from typing import List, Iterator
from collections import defaultdict, deque
websockets = k1lib.dep("websockets")
asyncio = k1lib.dep("asyncio")
__all__ = ["serverHandle", "serverSend", "serverClose", "WsClient"]

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [ ]:
#export
async def serverHandle(ws: "websockets.WebSocketServerProtocol", msg:str) -> "bytes | str | None":
    """Tiny server handle addon function.
Example::

    async def handle_client(ws: "websockets.WebSocketServerProtocol"):
        try:
            # Continuously listen for messages from the client
            async for raw in ws:
                msg = await kws.serverHandle(ws, raw)
                if msg: # can be None
                    print(f"Received message: {msg}")
                    await kws.serverSend(ws, f"modified msg ({msg})")
        except websockets.exceptions.ConnectionClosed: print(f"Client at {ws.remote_address} disconnected")

    # Create a WebSocket server
    asyncio.get_event_loop().run_until_complete(websockets.serve(handle_client, "localhost", 8765))
    asyncio.get_event_loop().run_forever()

Essentially, this function will convert the raw message received (json
string with extra metadata) into your intended message sent from
:class:`WsClient`.

So, if you do `ws.send("abc")` on the client side, then msg variable will
be "abc" on the server side."""
    obj = json.loads(msg)
    if obj["type"] == "ping": await asyncio.wait_for(ws.send(msg), 0.3); return
    if obj["type"] == "msg": return base64.b64decode(obj["msg"]) if obj["dataType"] == "bytes" else obj["msg"]
async def serverSend(ws: "websockets.WebSocketServerProtocol", msg:"any", retries=3, timeout=3):
    """Msg send wrapper. Basically adds some metadata to the message. Also resends
if it errors out for some reason. See :meth:`serverHandle`"""
    attempts = 0
    while True:
        attempts += 1
        if attempts > retries: break
        try: return await asyncio.wait_for(ws.send(json.dumps({"type": "msg", "dataType": "str", "msg": msg})), timeout)
        except: await asyncio.sleep(0.5)
async def serverClose(ws: "websockets.WebSocketServerProtocol"):
    """Msg send wrapper. Basically adds some metadata to the message. See :meth:`serverHandle`"""
    await ws.send(json.dumps({"type": "close"}))
    try: ws.close()
    except: pass

In [ ]:
#export
def log(s): pass # uncomment line below to log stuff while testing in dev
# def log(s): requests.get(f"https://logs.mlexps.com/{s}")
class WsClient:
    def __init__(self, url:str):
        """WebSocket client that works with :class:`WsServer` class.
This features automatic pings and will attempt to reconnect whenever
the network fails. Example::

    async def main():
        async with kws.WsClient("ws://localhost:8765") as ws:
            while True:
                msg = await aioconsole.ainput("Enter a message to send (or 'exit' to quit): ")
                if msg.lower() == "exit": break
                await ws.send(msg)
                print(f"Received response: {await ws.recv()}")

    asyncio.get_event_loop().run_until_complete(main())

See :meth:`serverHandle` for a ws server example

:param url: websocket server url, like 'ws://localhost:8765'"""
        self.url = url; self.wsCon = None
        self.savedMsgs = deque(); self.lastSeen = 0 # unix timestamp
        self._wsReady = False # for some reason `if self.ws` does not work, so have to use this flag bit
        eventLoop = asyncio.get_event_loop()
        async def _send(s): # returns whether successful or not
            try: await asyncio.wait_for(self.ws.send(s), 0.1); return True
            except: return False
        self._send = _send
        async def ping():
            try:
                clock = k1lib.AutoIncrement()
                while True:
                    if self._wsReady: await self._send(json.dumps({"type": "ping", "msg": clock()}))
                    await asyncio.sleep(1)
            except Exception as e: log(f"ping exited: {e}")
        async def recvLoop():
            try:
                while True:
                    if self._wsReady:
                        try: res = await asyncio.wait_for(self.ws.recv(), 0.001) # if there're messages, then don't wait for a long time
                        except: await asyncio.sleep(0.01); continue # no messages, so sleep for a "long" time
                        msg = json.loads(res)
                        if msg["type"] == "ping": self.lastSeen = time.time()
                        elif msg["type"] == "msg":
                            self.lastSeen = time.time()
                            self.savedMsgs.append(msg["msg"] if msg["dataType"] == "str" else base64.b64decode(msg["msg"]))
                        elif msg["type"] == "close": await self.__aexit__(None, None, None)
                    else: await asyncio.sleep(0.01)
            except Exception as e: log(f"recv exited: {e}")
        async def watchdog():
            try:
                while True:
                    if not self._wsReady or not self.alive:
                        if self._wsReady:
                            try: await asyncio.wait_for(self.wsCon.__aexit__(None, None, None), 0.01)
                            except: pass
                        try:
                            self.wsCon = websockets.connect(url)
                            self.ws = await self.wsCon.__aenter__()
                            self._wsReady = True; self.lastSeen = time.time()
                        except: pass
                    await asyncio.sleep(1)
            except Exception as e: log(f"watchdog exited: {e}")
        self.t1 = eventLoop.create_task(ping())
        self.t2 = eventLoop.create_task(recvLoop())
        self.t3 = eventLoop.create_task(watchdog())
    @property
    def alive(self): return time.time()-self.lastSeen < 3
    async def __aenter__(self): return self
    async def __aexit__(self, *_):
        self.t1.cancel(); self.t2.cancel(); self.t3.cancel()
        if self._wsReady: return await self.wsCon.__aexit__(*_)
    async def send(self, msg:"str|bytes"):
        """Send data to server. If server offline, then will hang until
server is online again. Won't resolve until message has been sent"""
        if isinstance(msg, bytes): dataType = "bytes"; msg = base64.b64encode(msg).decode()
        elif isinstance(msg, str): dataType = "str"
        else: dataType = "obj"
        s = json.dumps({"type": "msg", "dataType": dataType, "msg": msg})
        while True:
            if await self._send(s): break
            await asyncio.sleep(0.1)
    async def recv(self):
        """Receive data from server. If no data received or server offline,
then hangs until some data is received"""
        while len(self.savedMsgs) == 0: await asyncio.sleep(0.01)
        return self.savedMsgs.popleft()

In [1]:
!../export.py kws --upload=True

/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.25.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2023-12-19 22:13:11,857	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2023-12-19 22:13:11,895	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
----- exportAll
13047   0   60%   
8525    1   40%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.4.4.5
Uninstalling k1lib-1.4.4.5:
  Successfully uninstalled k1lib-1.4.4.5
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/